In [7]:
# directory kiểm thử: 'Test data/'
# cấu trúc: trong thư mục 'Test data/' gồm các file 1.mp4/mov...
# trả về file csv 2 cột: 1 cột là tên video, 1 cột là kết quả dự đoán
# cột 1: "video", cột 2 "dự đoán"

import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
import json
import os

directory = 'Test - vòng loại/' # đường dẫn đến thư mục kiểm thử

label_list = {0: 'barbell biceps curl',
              1: 'bench press',
              2: 'chest fly machine', 
              3: 'deadlift', 
              4: 'decline bench press', 
              5: 'hammer curl', 
              6: 'hip thrust', 
              7: 'incline bench press', 
              8: 'lat pulldown', 
              9: 'lateral raise', 
              10: 'leg extension', 
              11: 'leg raises', 
              12: 'plank', 
              13: 'pull Up', 
              14: 'push-up', 
              15: 'romanian deadlift', 
              16: 'russian twist', 
              17: 'shoulder press', 
              18: 'squat', 
              19: 't bar row', 
              20: 'tricep dips', 
              21: 'tricep Pushdown'}

n_frames = 20
pose = mp.solutions.pose.Pose()
model = tf.keras.models.load_model("model_remake_LSTM_4_44-88-44-22.keras")

list_test = os.listdir(directory)
result_test = {}

In [8]:
# đọc từng video, tách frame, xác định khung xương, gom nhóm và xác định nhãn

def calculate_bounding_box(landmarks):
    min_x = min(landmark.x for landmark in landmarks)
    min_y = min(landmark.y for landmark in landmarks)
    max_x = max(landmark.x for landmark in landmarks)
    max_y = max(landmark.y for landmark in landmarks)

    return BoundingBox(min_x, min_y, max_x, max_y)

class BoundingBox:
    def __init__(self, top, left, bottom, right):
        self.top = top
        self.left = left
        self.bottom = bottom
        self.right = right

def recalculate_landmarks(landmarks, bounding_box):
    for landmark in landmarks:
        landmark.x = (landmark.x - bounding_box.top)/(bounding_box.bottom - bounding_box.top)
        landmark.y = (landmark.y - bounding_box.left)/(bounding_box.right - bounding_box.left)
    return landmarks

def make_landmark_timestep(landmarks):
    c_lm = []
    for landmark in landmarks:
        c_lm.append(landmark.x)
        c_lm.append(landmark.y)
        c_lm.append(landmark.z)
        c_lm.append(landmark.visibility)
    return c_lm

def detect(model, lm_list, label_list):
    lm_list = np.array(lm_list)
    lm_list = np.expand_dims(lm_list, axis=0)
    results = model.predict(lm_list,verbose=0)
    return results

for file in list_test:
    print("processing " + file)
    file_path = os.path.join(directory,file)
    cap = cv2.VideoCapture(file_path)
    lm_list = []
    temp_label_list = np.zeros((1,22))
    while cap.isOpened():
        success, img = cap.read()
        if not success:
            break
        # img = cv2.resize(img, (1280, 720)) # resize về 1280x720
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = pose.process(imgRGB)
        if result.pose_landmarks:
            landmarks = result.pose_landmarks.landmark
            bounding_box = calculate_bounding_box(landmarks)
            resized_landmarks = recalculate_landmarks(landmarks, bounding_box)
            lm_list.append(make_landmark_timestep(resized_landmarks))
            if len(lm_list) == n_frames: # detect sau khi gom được n_frames
                predict = detect(model, lm_list, label_list)
                temp_label_list += predict
                lm_list = []
    file_label = np.argmax(temp_label_list,axis=1)
    result_test[file] = label_list[file_label[0]]
    cap.release()
        
print("Processed all tests")

processing barbell biceps curl_1.mp4
processing bench press_1.mp4
processing bench press_2.mp4
processing chest fly machine_1.mp4
processing chest fly machine_2.mp4
processing chest fly machine_3.mp4
processing deadlift_1.mp4
processing declince bench press_1.mp4
processing decline bench press_2.mp4
processing hammer curl_1.mp4
processing hammer curl_2.mp4
processing hip thrush_1.mp4
processing hip thrust_1.mp4
processing incline bench press_1.mp4
processing incline bench press_2.mp4
processing lat pulldown_1.mp4
processing lat pulldown_2.mp4
processing lateral raise_1.mp4
processing leg extension_1.mp4
processing leg extension_2.mp4
processing leg raises_1.mp4
processing leg raises_2.mp4
processing plank_1.mp4
processing pull up_1.mp4
processing pull up_2.mp4
processing push-up_1.mp4
processing romanian deadlift_1.mp4
processing romanian deadlift_2.mp4
processing russian twist_1.mp4
processing russian twist_2.mp4
processing sholder press_2.mp4
processing shoulder press_1.mp4
processin

In [10]:
# viết ra file csv:
import csv
with open("du_doan.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["video","Dự đoán"])
    for key, value in result_test.items():
        writer.writerow([key, value])